In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def imagej_hist(src):
    maximum = np.max(src)
    minimum = np.min(src)
    scale_map = np.linspace(minimum, maximum, num=2**8, endpoint=False)
    scale_map = np.append(scale_map, maximum)
    hist = []
    for i in range(2**8):
        up = len(np.where(src>=scale_map[i])[0])
        down = len(np.where(src<scale_map[i+1])[0])
        hist.append(up + down - src.size)
    return hist

def auto_br(input_src):
    """
    Copy from imageJ
    """
    src = input_src
    h, w = src.shape[:2]
    src_min = np.min(src)
    src_max = np.max(src)
    pixel_count = h * w
    hist = imagej_hist(src)
    th = pixel_count/5000
    limit = pixel_count/10
    bin_size = (src_max - src_min) / 2**8
    hmin = 0
    hmax = 2**8-1
    for i in range(len(hist)):
        count = hist[i]
        count = 0 if count>limit else count
        if count > th:
            hmin = i
            break
    for i in range(len(hist)):
        count = hist[2**8-1 - i]
        count = 0 if count>limit else count
        if count > th:
            hmax = 2**8-1 -i
            break
    tgt_min = src_min + hmin * bin_size
    tgt_max = src_min + hmax * bin_size
    tgt_img = ((src - tgt_min) * 2**16/(tgt_max - tgt_min))
    tgt_img[np.where(tgt_img <= 0)] = 0
    tgt_img[np.where(tgt_img >= 2**16 - 1)] = 2**16 - 1
    tgt_img = tgt_img.astype(np.uint16)
    return tgt_img

In [ ]:
from matplotlib.pyplot import figure
from pathlib import Path
plt.rcParams["text.usetex"] = True
plt.rcParams["font.size"] = 30
figure(figsize=(8, 16))
src = cv2.imread("./test.tif", -1)[512:1536, 512:1536][750:950, 50:250]
tgt = auto_br(src)
plt.subplot(2,1,1)
plt.hist(src.ravel(), 256, [0, 65536])
plt.xlabel(" \n \n")
plt.subplot(2,1,2)
plt.hist(tgt.ravel(), 256, [0, 65536])
plt.savefig("./imgs/hist.jpg")
plt.show()